In [2]:
import folium
import pandas as pd
import json
from branca.colormap import linear

# Load the CSV files
taxi_y_cleaned_path = '../data/taxi_y_cleaned.csv'
taxi_zone_lookup_path = '../data/taxi+_zone_lookup.csv'
taxi_y_cleaned = pd.read_csv(taxi_y_cleaned_path)
taxi_zone_lookup = pd.read_csv(taxi_zone_lookup_path)

# Merge the taxi trip data with the zone lookup data to get the zone names for pickup locations
taxi_data_merged = taxi_y_cleaned.merge(taxi_zone_lookup, how='left', left_on='pulocationid', right_on='LocationID')

# Extract hour from the 'tpep_pickup_datetime' for analysis of busy times
taxi_data_merged['tpep_pickup_datetime'] = pd.to_datetime(taxi_data_merged['tpep_pickup_datetime'])
taxi_data_merged['pickup_hour'] = taxi_data_merged['tpep_pickup_datetime'].dt.hour

# Group by pickup location and hour to find the count of pickups for each hour and location
pickup_counts_by_hour = taxi_data_merged.groupby(['pulocationid', 'pickup_hour']).size().reset_index(name='pickup_count')

# Load the GeoJSON file
geojson_path = '../data/NYC Taxi Zones.geojson'
with open(geojson_path, 'r') as file:
    taxi_zones_geojson = file.read()
taxi_zones_geojson_dict = json.loads(taxi_zones_geojson)

# Initialize a new folium map
folium_map_final = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

# Create a colormap
colormap = linear.YlOrRd_09.scale(0, pickup_counts_by_hour['pickup_count'].max())

# Function to calculate color for each feature
def calculate_color_updated(location_id, hour):
    pickup_count = pickup_counts_by_hour[
        (pickup_counts_by_hour['pulocationid'] == location_id) &
        (pickup_counts_by_hour['pickup_hour'] == hour)
    ]['pickup_count'].values
    pickup_count = pickup_count[0] if len(pickup_count) > 0 else 0  # Default to 0 if no data available
    color = colormap(pickup_count)
    return color

# Function to apply style to each feature
def apply_style(feature):
    location_id = feature['properties']['location_id']  # Corrected to access 'location_id' in 'properties'
    color = calculate_color_updated(location_id, 8)  # Calculate color for 8 AM
    return {
        'fillColor': color,
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.5
    }

# Function to highlight GeoJSON features
def highlight_function(feature):
    return {
        'fillColor': 'red',
        'color': 'red',
        'weight': 3,
        'fillOpacity': 0.7
    }

# Add the GeoJSON features to the map with the correct style and highlighting
for feature in taxi_zones_geojson_dict['features']:
    folium.GeoJson(
        feature,
        style_function=apply_style,  # Apply the style to each feature
        highlight_function=highlight_function  # Apply the highlight style
    ).add_to(folium_map_final)

# Add the colormap to the map
colormap.caption = "Number of Taxi Pickups"
colormap.add_to(folium_map_final)

# Save and show the map
folium_map_final_path = '../data/taxi_pickups_map_final.html'
folium_map_final.save(folium_map_final_path)


In [ ]:
import folium
print(folium.__version__)


In [5]:
import folium
import pandas as pd
import json
from folium.plugins import TimeSliderChoropleth
from branca.colormap import linear

# Load the CSV files
taxi_y_cleaned_path = '../data/taxi_y_cleaned.csv'
taxi_zone_lookup_path = '../data/taxi+_zone_lookup.csv'
taxi_y_cleaned = pd.read_csv(taxi_y_cleaned_path)
taxi_zone_lookup = pd.read_csv(taxi_zone_lookup_path)

# Merge the taxi trip data with the zone lookup data to get the zone names for pickup locations
taxi_data_merged = taxi_y_cleaned.merge(taxi_zone_lookup, how='left', left_on='pulocationid', right_on='LocationID')

# Extract hour from the 'tpep_pickup_datetime' for analysis of busy times
taxi_data_merged['tpep_pickup_datetime'] = pd.to_datetime(taxi_data_merged['tpep_pickup_datetime'])
taxi_data_merged['pickup_hour'] = taxi_data_merged['tpep_pickup_datetime'].dt.hour

# Group by pickup location and hour to find the count of pickups for each hour and location
pickup_counts_by_hour = taxi_data_merged.groupby(['pulocationid', 'pickup_hour']).size().reset_index(name='pickup_count')

# Load the GeoJSON file
geojson_path = '../data/NYC Taxi Zones.geojson'
with open(geojson_path, 'r') as file:
    taxi_zones_geojson = file.read()
taxi_zones_geojson_dict = json.loads(taxi_zones_geojson)

# Initialize a new folium map
folium_map_final = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

# Create a colormap
colormap = linear.YlOrRd_09.scale(0, pickup_counts_by_hour['pickup_count'].max())

# Function to calculate color for each feature
def calculate_color_updated(location_id, hour):
    pickup_count = pickup_counts_by_hour[
        (pickup_counts_by_hour['pulocationid'] == location_id) &
        (pickup_counts_by_hour['pickup_hour'] == hour)
    ]['pickup_count'].values
    pickup_count = pickup_count[0] if len(pickup_count) > 0 else 0  # Default to 0 if no data available
    return colormap(pickup_count)

# Prepare the data for TimeSliderChoropleth
# Create a base date for the timestamps
base_date = pd.Timestamp('2023-01-01')
styledict = {
    str(feature['properties']['location_id']): {
        (base_date + pd.Timedelta(hours=time)).value // 10**6: {  # Convert to Unix timestamp in milliseconds
            'color': calculate_color_updated(feature['properties']['location_id'], time),
            'opacity': 0.7,
            'weight': 0.5,
            'fillColor': calculate_color_updated(feature['properties']['location_id'], time)
        } for time in range(24)  # Assuming we have hourly data
    } for feature in taxi_zones_geojson_dict['features']
}

# Add the TimeSliderChoropleth to the map
slider = TimeSliderChoropleth(
    data=taxi_zones_geojson_dict,
    styledict=styledict
).add_to(folium_map_final)

# Add the colormap to the map
colormap.caption = "Number of Taxi Pickups"
colormap.add_to(folium_map_final)

# Save and show the map
folium_map_final_path = '../data/taxi_pickups_map_final2.html'
folium_map_final.save(folium_map_final_path)


KeyError: 'LocationID'